<a href="https://colab.research.google.com/github/olgasem10/Compling-tasks/blob/master/Hometask2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Триграммная модель


In [132]:
!wget https://raw.githubusercontent.com/olgasem10/Compling-tasks/master/%D0%91%D0%B5%D0%BB%D0%B0%D1%8F%20%D0%B3%D0%B2%D0%B0%D1%80%D0%B4%D0%B8%D1%8F.txt

--2019-10-22 12:57:56--  https://raw.githubusercontent.com/olgasem10/Compling-tasks/master/%D0%91%D0%B5%D0%BB%D0%B0%D1%8F%20%D0%B3%D0%B2%D0%B0%D1%80%D0%B4%D0%B8%D1%8F.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 888054 (867K) [text/plain]
Saving to: ‘Белая гвардия.txt’

Белая гвардия.txt   100%[===================>] 867.24K  --.-KB/s    in 0.05s   

2019-10-22 12:57:56 (16.5 MB/s) - ‘Белая гвардия.txt’ saved [888054/888054]



In [0]:
bulgakov = open('Белая гвардия.txt').read()

Задаем функцию для нормализации:

In [0]:
from string import punctuation
punctuation += '«»—–…“”'


def normalize(text):
    normalized_text = [word.strip(punctuation) for word in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

Задаем функцию для выявления нграмм:

In [0]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

Делим текст на предложения и добавляем теги начала и конца предложения:

In [0]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
sentences = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(bulgakov)]


Создаем списки униграм, биграм и триграм:

In [0]:
from collections import Counter

In [0]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()
for sentence in sentences:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence, 2))
    trigrams.update(ngrammer(sentence, 3))

Чтобы в дальнейшем сравнить результаты работы биграммной и триграммной модели, сначала реализуем биграммную модель

Создаем матрицу вероятностей перейти из одного слова в другое:

In [0]:
import numpy as np

In [0]:
matrix = np.zeros((len(unigrams), 
                   len(unigrams)))
id2word = list(unigrams)
word2id = {word:i for i, word in enumerate(id2word)}


for ngram in bigrams:
    word1, word2 = ngram.split()
    matrix[word2id[word1]][word2id[word2]] = (bigrams[ngram]/unigrams[word1])

Задаем генерирующую функцию:

In [0]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

Генерируем текст:

In [142]:
print(generate(matrix, id2word, word2id).replace('<end>', '\n'))

огне-е-е-е-е-е-ем зазвенело в тысяча человек водрузивший сверх серого неба продырявленного черным и слабо но вся суть 
 за его 
 дай мне туда поехать 
 студзинский залился густейшей краской 
 в руки и сон стал 
 паровоз нес и задохнешься 
 зачем это доктор алексей турбин прищурившись огляделся и желтые длинные пульманы окна 
 братики сестрички обратите внимание на лариосиковой стороне от анюты николка 
 вовсе 
 грудь 
 зачем турбин больше не слушал 
 с утра сегодня что ты боже мой 
 формируется армия которая взрывами доносилась откуда-то со времен крестовых походов службы и не подвергать опасности себя вину


Теперь реализуем триграммную модель

Создаем матрицу вероятностей перейти от биграммы к следующему слову

In [0]:
trimatrix = np.zeros((len(bigrams), len(unigrams)))
id2word = list(unigrams)
word2id = {word:i for i, word in enumerate(id2word)}
id2bigram = list(bigrams)
bigram2id = {bigram:i for i, bigram in enumerate(id2bigram)}

for ngram in trigrams:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    trimatrix[bigram2id[bigram]][word2id[word3]] = (trigrams[ngram]/bigrams[bigram]) 

Отдельно задаем матрицу вероятностей перейти из тега start к следующему слову:

In [0]:
startmatrix = np.zeros((1, len(unigrams)))
id2word = list(unigrams)
word2id = {word:i for i, word in enumerate(id2word)}


for ngram in bigrams:
    word1, word2 = ngram.split()
    if word1 == '<start>':
        startmatrix[word2id[word1]][word2id[word2]] = (bigrams[ngram]/unigrams[word1])

Задаем генерирующую функцию:

In [0]:
def trigenerate(trimatrix, startmatrix, id2word, word2id, bigram2id, n=100, start='<start>'):
    text = []
    previous_idx = 0
    last_idx = word2id[start]
    
    for i in range(n):
        if last_idx == word2id[start]:
            chosen = np.random.choice(startmatrix.shape[1], p=startmatrix[last_idx])
            text.append(id2word[chosen])
            previous_idx = last_idx
            last_idx = chosen
        current_bigram = id2word[previous_idx] + ' ' + id2word[last_idx]
        current_idx = bigram2id[current_bigram]
        chosen = np.random.choice(trimatrix.shape[1], p=trimatrix[current_idx])
        text.append(id2word[chosen])
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        previous_idx = last_idx
        last_idx = chosen
    
    return ' '.join(text)

In [147]:
print(trigenerate(trimatrix, startmatrix, id2word, word2id, bigram2id).replace('<end>', '\n'))

и они с красными бантами 
 так вот спрашиваю как по-украински кот 
 и в частности машинным маслом и керосином наилучшим образом были смазаны и най-турсов кольт и вскрытая кобура била и хлопала его по спине николки и лариосика бесшумно шаркавших накануне 
 один маленький укладистый и действительно в высшей степени живые и смышленые глаза смотрели явно устало но внимательно 
 когда он лучше меня знает 
 как это страшно они самое меня чуть не застрелили 
 настала тишина и только тут поморгав глазами заметил погоны турбина 
 большевик 
 брынь раненый говорил ласково и утвердительно закачал головой 
 а где же сенегальцев роты 
 убедившись что улица окончательно затихла не


Сравнивая результаты работы биграммной и триграммной моделей, можно отметить, что тексты, сгенерированные с помощью триграмм в целом выглядят более осмысленными. В основном это видимо достигается за счет того, что триграммная модель, опираясь на два предыдущих слова, а не на одно, способна воспроизводить более протяженные отрывки из оригинального текста, чем это удается биграммной модели

# Byte-pair-encoding

In [2]:
!wget https://raw.githubusercontent.com/olgasem10/Compling-tasks/master/%D0%A7%D0%B5%D1%80%D0%B5%D0%BF%D0%B0%D1%85%D0%B0.txt

--2019-10-22 10:49:08--  https://raw.githubusercontent.com/olgasem10/Compling-tasks/master/%D0%A7%D0%B5%D1%80%D0%B5%D0%BF%D0%B0%D1%85%D0%B0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4132 (4.0K) [text/plain]
Saving to: ‘Черепаха.txt’

Черепаха.txt        100%[===================>]   4.04K  --.-KB/s    in 0s      

2019-10-22 10:49:09 (70.1 MB/s) - ‘Черепаха.txt’ saved [4132/4132]



In [3]:
text = open('Черепаха.txt').read()
text

'Редкая морская черепаха появилась в «Москвариуме» на ВДНХ.\nУвидеть черепаху в общем аквариуме можно ежедневно. А по понедельникам, средам и пятницам гости смогут также понаблюдать за ее кормлением.\nГости «Москвариума» на ВДНХ увидят редкую морскую черепаху бисса, занесенную в Международную Красную книгу.\n\nСамец был передан одним из столичных зоомагазинов. Посетители уже назвали его Барбарис.\n\n«Символично, что именно во Всемирный день черепахи в главном морском аквариуме появился новый обитатель из редкого рода морских черепах бисса, — рассказал ихтиолог океанариума Ирина Мейнцер. — В “Москвариуме” Барбарис получил новый дом — большой морской аквариум, где он может расти и развиваться в самой благоприятной среде».\n\nЧерепаха была ослаблена, ихтиологи помогали ей набрать вес и укрепить иммунитет, прежде чем выпустить в общий аквариум. В неделю Барбарис съедает около 3,5 килограмма корма. Его излюбленное лакомство — кальмары, а также креветки и рыба. Сейчас вес черепахи увеличился

In [0]:
import nltk
from nltk.tokenize import sent_tokenize

In [148]:
sentences=sent_tokenize(text, 'russian')
sentences

['Редкая морская черепаха появилась в «Москвариуме» на ВДНХ.',
 'Увидеть черепаху в общем аквариуме можно ежедневно.',
 'А по понедельникам, средам и пятницам гости смогут также понаблюдать за ее кормлением.',
 'Гости «Москвариума» на ВДНХ увидят редкую морскую черепаху бисса, занесенную в Международную Красную книгу.',
 'Самец был передан одним из столичных зоомагазинов.',
 'Посетители уже назвали его Барбарис.',
 '«Символично, что именно во Всемирный день черепахи в главном морском аквариуме появился новый обитатель из редкого рода морских черепах бисса, — рассказал ихтиолог океанариума Ирина Мейнцер.',
 '— В “Москвариуме” Барбарис получил новый дом — большой морской аквариум, где он может расти и развиваться в самой благоприятной среде».',
 'Черепаха была ослаблена, ихтиологи помогали ей набрать вес и укрепить иммунитет, прежде чем выпустить в общий аквариум.',
 'В неделю Барбарис съедает около 3,5 килограмма корма.',
 'Его излюбленное лакомство — кальмары, а также креветки и рыба.'

Разделим полученные предложения на символы, удалим знаки препинания

In [0]:
symbols = [list(sentence.lower()) for sentence in sentences]
clean_symbols = []
for sent in symbols:
  a = [symbol.strip(punctuation) for symbol in sent]
  a = [symbol for symbol in a if symbol]
  clean_symbols.append(a)

In [0]:
import gensim

Первая итерация

In [0]:
ph = gensim.models.Phrases(clean_symbols, min_count=1, threshold=1)
p = gensim.models.phrases.Phraser(ph)

Зададим функцию, с помощью которой можно будет отслеживать, какие сочетания объединились на каждом шаге

In [0]:
def add(mylist, n):
  list1 = []
  for sent in mylist:
    list2 = [s for s in sent if len(s) >= n]
    list1.extend(list2)
  list3 = []
  for i in list1:
        if i not in list3:
          list3.append(i)
  return list3

После первого шага объединились следующие сочетания букв:

In [94]:
add(p[clean_symbols], 2)[:20]

['с_к',
 'ч_е',
 'а_х',
 'у_м',
 'т_ь',
 'б_щ',
 'к_в',
 'ж_е',
 'л_ь',
 'г_о',
 'о_г',
 'к_ж',
 'л_ю',
 'у_ю',
 'ж_д',
 'б_ы',
 'н_ы',
 'й_ ',
 'в_ы',
 'н_ц']

Вторая итерация

In [120]:
ph2 = gensim.models.Phrases(p[clean_symbols], min_count=1, threshold=1)
p2 = gensim.models.phrases.Phraser(ph2)
add(p2[p[clean_symbols]], 4)[:20]

['р_с_к',
 ' _ч_е',
 'п_а_х',
 'с_к_в',
 'и_у_м',
 'т_ь_ ',
 'ч_е_р',
 'о_б_щ',
 'а_к_в',
 'ж_е_д',
 'е_л_ь',
 'о_г_у',
 'а_к_ж',
 'б_л_ю',
 'а_т_ь',
 'у_ю_ ',
 'н_у_ю',
 'е_ж_д',
 'б_ы_л',
 'н_ы_х']

Третья итерация

In [127]:
ph3 = gensim.models.Phrases(p2[p[clean_symbols]], min_count=1, threshold=1)
p3 = gensim.models.phrases.Phraser(ph3)
add(p3[p2[p[clean_symbols]]], 6)[:20]

['я_ _ч_е',
 'с_к_в_а',
 'р_и_у_м',
 'ч_е_р_е',
 'п_а_х_у',
 ' _о_б_щ',
 ' _а_к_в',
 'е_ж_е_д',
 'е_л_ь_н',
 'г_о_с_т',
 'м_о_г_у',
 'т_а_к_ж',
 'а_б_л_ю',
 'д_а_т_ь',
 'н_у_ю_ ',
 ' _б_ы_л',
 'н_ы_х_ ',
 'р_н_ы_й_ ',
 ' _ч_е_р',
 'е_п_а_х']

Четвертая итерация

In [128]:
ph4 = gensim.models.Phrases(p3[p2[p[clean_symbols]]], min_count=1, threshold=1)
p4 = gensim.models.phrases.Phraser(ph4)
add(p4[p3[p2[p[clean_symbols]]]], 8)[:20]

['м_о_р_с_к',
 'я_ _ч_е_р_е',
 'с_к_в_а_р_и_у_м',
 'ч_е_р_е_п_а_х_у',
 'м_ _а_к_в',
 'а_р_и_у_м',
 ' _е_ж_е_д',
 'е_д_е_л_ь_н',
 ' _п_я_т_н',
 'г_о_с_т_и',
 'м_о_г_у_т',
 ' _т_а_к_ж',
 'н_а_б_л_ю',
 'д_а_т_ь_ _з_а',
 'н_у_ю_ _к',
 'п_о_с_е_т',
 'р_н_ы_й_ ',
 ' _ч_е_р_е_п_а_х',
 ' _н_о_в_ы',
 'и_о_л_о_г']

На этом шаге впервые уже можно выявить несколько целых слов - черепаху, гости, могут, был, его, аквариум, очень, может

Пятая итерация

In [126]:
ph5 = gensim.models.Phrases(p4[p3[p2[p[clean_symbols]]]], min_count=1, threshold=1)
p5 = gensim.models.phrases.Phraser(ph5)
add(p5[p4[p3[p2[p[clean_symbols]]]]], 8)[:20]

['м_о_р_с_к',
 'я_ _ч_е_р_е',
 ' _в_ _м_о',
 'с_к_в_а_р_и_у_м',
 'ч_е_р_е_п_а_х_у',
 'м_ _а_к_в_а_р_и_у_м',
 ' _е_ж_е_д_н',
 ' _п_о_н_е_д_е_л_ь_н',
 ' _с_р_е_д',
 'и_ _п_я_т_н',
 'г_о_с_т_и',
 'м_о_г_у_т',
 ' _т_а_к_ж',
 'н_а_б_л_ю_д_а_т_ь_ _з_а',
 'е_ _к_о_р',
 'м_л_е_н_и_е',
 ' _м_о_с_к_в_а_р_и_у_м',
 ' _б_и_с_с',
 'н_у_ю_ _к',
 'п_о_с_е_т_и_т_е']

На этом шаге, кроме новых слов можно отметить также появление словосочетания - наблюдать за, к тому же некоторые слова присоединяют к себе начала или окончания соседних слов